In [1]:
# Change coordinate txt to csv
from shutil import copyfile
copyfile('images/image_coords.txt', 'images/image_coords.csv')

'images/image_coords.csv'

In [13]:
import csv
import pickle
from PIL import Image
import numpy as np

input_images = []
with open('images/image_coords.csv', 'r') as coord_file:
    reader = csv.reader(coord_file)
    for row in reader:
        try:
            with open('images/' + row[0]) as f:
                im = Image.open('images/' + row[0], 'r').convert('L')
                pix = im.load()
                im_array = []
                for i in range(im.size[0]):
                    for j in range(im.size[1]):
                        im_array.append(pix[i,j])
                im_array = np.expand_dims(np.array(im_array), axis=0)
                input_images.append(im_array)
                print('images/' + row[0])
        except FileNotFoundError:
            continue

images/0_61.png
images/0_74.png
images/0_75.png
images/0_76.png
images/0_77.png
images/0_78.png
images/0_79.png
images/0_80.png
images/0_81.png
images/0_82.png
images/0_83.png
images/0_84.png
images/0_85.png
images/0_86.png
images/0_87.png
images/0_88.png
images/0_89.png
images/0_90.png
images/0_97.png
images/0_98.png
images/0_99.png
images/0_100.png
images/0_101.png
images/0_102.png
images/0_103.png
images/0_104.png
images/0_105.png
images/0_106.png
images/0_107.png
images/0_108.png
images/0_109.png
images/0_110.png
images/0_111.png
images/0_112.png
images/0_113.png
images/1_33.png
images/1_34.png
images/1_35.png
images/1_65.png
images/1_66.png
images/1_67.png
images/1_68.png
images/1_69.png
images/1_70.png
images/1_71.png
images/1_72.png
images/1_73.png
images/1_74.png
images/1_75.png
images/1_76.png
images/1_77.png
images/1_78.png
images/1_79.png
images/1_80.png
images/1_81.png
images/1_82.png
images/1_83.png
images/1_84.png
images/1_85.png
images/1_86.png
images/1_87.png
images/1_8

images/9_3.png
images/9_4.png
images/9_5.png
images/9_6.png
images/9_7.png
images/9_8.png
images/9_9.png
images/9_10.png
images/9_11.png
images/9_32.png
images/9_33.png
images/9_34.png
images/9_35.png
images/9_36.png
images/9_37.png
images/9_38.png
images/9_39.png
images/9_40.png
images/9_41.png
images/9_42.png
images/9_43.png
images/9_44.png
images/9_45.png
images/9_46.png
images/9_47.png
images/9_48.png
images/9_49.png
images/9_50.png
images/9_64.png
images/9_65.png
images/9_66.png
images/9_67.png
images/9_68.png
images/9_69.png
images/9_70.png
images/9_71.png
images/9_72.png
images/9_73.png
images/9_74.png
images/9_75.png
images/9_76.png
images/9_77.png
images/9_78.png
images/9_79.png
images/9_80.png
images/9_81.png
images/9_82.png
images/9_83.png
images/9_84.png
images/9_85.png
images/9_86.png
images/9_88.png
images/9_89.png
images/9_91.png
images/9_92.png
images/9_93.png
images/9_94.png
images/9_95.png
images/9_96.png
images/9_97.png
images/9_98.png
images/9_99.png
images/9_100.pn

images/14_93.png
images/14_94.png
images/14_95.png
images/14_96.png
images/14_98.png
images/14_100.png
images/14_101.png
images/14_102.png
images/14_103.png
images/14_104.png
images/14_105.png
images/14_106.png
images/14_107.png
images/14_108.png
images/14_109.png
images/14_110.png
images/14_111.png
images/14_112.png
images/14_113.png
images/15_2.png
images/15_3.png
images/15_4.png
images/15_5.png
images/15_6.png
images/15_7.png
images/15_8.png
images/15_9.png
images/15_11.png
images/15_12.png
images/15_13.png
images/15_22.png
images/15_23.png
images/15_24.png
images/15_25.png
images/15_26.png
images/15_27.png
images/15_28.png
images/15_29.png
images/15_30.png
images/15_31.png
images/15_32.png
images/15_33.png
images/15_34.png
images/15_35.png
images/15_36.png
images/15_37.png
images/15_38.png
images/15_39.png
images/15_40.png
images/15_41.png
images/15_42.png
images/15_43.png
images/15_44.png
images/15_45.png
images/15_46.png
images/15_47.png
images/15_48.png
images/15_49.png
images/1

images/19_57.png
images/19_58.png
images/19_59.png
images/19_60.png
images/19_61.png
images/19_62.png
images/19_63.png
images/19_64.png
images/19_65.png
images/19_66.png
images/19_67.png
images/19_68.png
images/19_69.png
images/19_70.png
images/19_71.png
images/19_72.png
images/19_73.png
images/19_74.png
images/19_75.png
images/19_76.png
images/19_77.png
images/19_78.png
images/19_79.png
images/19_80.png
images/19_81.png
images/19_82.png
images/19_83.png
images/19_84.png
images/19_85.png
images/19_86.png
images/19_87.png
images/19_88.png
images/19_89.png
images/19_90.png
images/19_91.png
images/19_92.png
images/19_93.png
images/19_94.png
images/19_95.png
images/19_96.png
images/19_97.png
images/19_98.png
images/19_99.png
images/19_100.png
images/19_101.png
images/19_102.png
images/19_103.png
images/19_104.png
images/19_105.png
images/19_106.png
images/19_107.png
images/19_108.png
images/19_109.png
images/19_110.png
images/19_111.png
images/19_112.png
images/19_113.png
images/20_0.png
i

images/24_90.png
images/24_91.png
images/24_92.png
images/24_93.png
images/24_94.png
images/24_95.png
images/24_96.png
images/24_97.png
images/24_98.png
images/24_99.png
images/24_100.png
images/24_101.png
images/24_102.png
images/24_103.png
images/24_104.png
images/24_105.png
images/24_106.png
images/24_107.png
images/24_108.png
images/24_109.png
images/24_110.png
images/24_111.png
images/24_112.png
images/24_113.png
images/24_114.png
images/24_115.png
images/24_116.png
images/24_117.png
images/24_118.png
images/25_21.png
images/25_25.png
images/25_26.png
images/25_27.png
images/25_28.png
images/25_38.png
images/25_39.png
images/25_40.png
images/25_41.png
images/25_42.png
images/25_43.png
images/25_44.png
images/25_45.png
images/25_46.png
images/25_47.png
images/25_48.png
images/25_49.png
images/25_50.png
images/25_51.png
images/25_52.png
images/25_53.png
images/25_54.png
images/25_55.png
images/25_56.png
images/25_57.png
images/25_58.png
images/25_59.png
images/25_60.png
images/25_61

...

images/53_69.png
images/53_70.png
images/53_71.png
images/53_72.png
images/53_73.png
images/53_74.png
images/53_75.png
images/53_76.png
images/53_77.png
images/53_78.png
images/53_79.png
images/53_80.png
images/53_81.png
images/53_82.png
images/53_83.png
images/53_84.png
images/53_85.png
images/53_86.png
images/53_87.png
images/53_88.png
images/53_89.png
images/53_90.png
images/53_91.png
images/53_92.png
images/53_93.png
images/53_94.png
images/53_95.png
images/53_96.png
images/53_97.png
images/53_98.png
images/53_99.png
images/53_100.png
images/53_101.png
images/53_102.png
images/53_103.png
images/53_104.png
images/53_105.png
images/53_106.png
images/53_107.png
images/53_108.png
images/53_109.png
images/53_110.png
images/53_111.png
images/53_112.png
images/53_113.png
images/53_114.png
images/53_115.png
images/53_116.png
images/53_117.png
images/53_118.png
images/54_45.png
images/54_46.png
images/54_47.png
images/54_48.png
images/54_49.png
images/54_50.png
images/54_51.png
images/54_52

images/59_108.png
images/59_109.png
images/59_110.png
images/59_111.png
images/59_112.png
images/59_113.png
images/59_114.png
images/59_115.png
images/59_116.png
images/59_117.png
images/59_118.png
images/60_46.png
images/60_47.png
images/60_48.png
images/60_50.png
images/60_51.png
images/60_52.png
images/60_53.png
images/60_54.png
images/60_55.png
images/60_56.png
images/60_57.png
images/60_58.png
images/60_59.png
images/60_60.png
images/60_61.png
images/60_62.png
images/60_63.png
images/60_64.png
images/60_65.png
images/60_66.png
images/60_67.png
images/60_68.png
images/60_69.png
images/60_70.png
images/60_71.png
images/60_72.png
images/60_73.png
images/60_74.png
images/60_75.png
images/60_76.png
images/60_77.png
images/60_78.png
images/60_79.png
images/60_80.png
images/60_81.png
images/60_82.png
images/60_83.png
images/60_84.png
images/60_85.png
images/60_86.png
images/60_87.png
images/60_88.png
images/60_89.png
images/60_90.png
images/60_91.png
images/60_92.png
images/60_93.png
ima

images/67_68.png
images/67_69.png
images/67_70.png
images/67_71.png
images/67_72.png
images/67_73.png
images/67_74.png
images/67_75.png
images/67_76.png
images/67_77.png
images/67_78.png
images/67_79.png
images/67_80.png
images/67_81.png
images/67_82.png
images/67_83.png
images/67_84.png
images/67_85.png
images/67_86.png
images/67_87.png
images/67_88.png
images/67_89.png
images/67_91.png
images/67_92.png
images/67_93.png
images/67_94.png
images/67_95.png
images/67_96.png
images/67_97.png
images/67_98.png
images/67_99.png
images/67_100.png
images/67_101.png
images/67_102.png
images/67_103.png
images/67_104.png
images/67_105.png
images/67_106.png
images/67_107.png
images/67_108.png
images/67_109.png
images/67_110.png
images/67_111.png
images/67_112.png
images/67_113.png
images/67_114.png
images/67_115.png
images/67_116.png
images/67_117.png
images/67_118.png
images/68_29.png
images/68_56.png
images/68_57.png
images/68_58.png
images/68_59.png
images/68_60.png
images/68_61.png
images/68_62

images/74_96.png
images/74_97.png
images/74_98.png
images/74_99.png
images/74_100.png
images/74_101.png
images/74_102.png
images/74_103.png
images/74_104.png
images/74_105.png
images/74_106.png
images/74_107.png
images/74_108.png
images/74_109.png
images/74_110.png
images/74_111.png
images/74_112.png
images/74_113.png
images/74_114.png
images/74_115.png
images/74_116.png
images/74_117.png
images/74_118.png
images/75_56.png
images/75_57.png
images/75_58.png
images/75_59.png
images/75_60.png
images/75_61.png
images/75_62.png
images/75_63.png
images/75_64.png
images/75_65.png
images/75_66.png
images/75_67.png
images/75_68.png
images/75_69.png
images/75_70.png
images/75_71.png
images/75_72.png
images/75_73.png
images/75_74.png
images/75_75.png
images/75_76.png
images/75_77.png
images/75_78.png
images/75_79.png
images/75_80.png
images/75_81.png
images/75_82.png
images/75_83.png
images/75_84.png
images/75_85.png
images/75_86.png
images/75_87.png
images/75_88.png
images/75_89.png
images/75_90

MemoryError: 

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.01, shape=shape))

In [ ]:
def conv2d(x, W, stride_size=2):
    return tf.nn.conv2d(x, W, strides=[1,stride_size,stride_size,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
WIDTH = 640 # pixel width of each image
HEIGHT = 640 # pixel height of each image

x = tf.placeholder(tf.float32, shape=[None, WIDTH*HEIGHT]) # input is flattened image
y = tf.placeholder(tf.float32, shape=[None, 1]) # output is one value

x_image = tf.reshape(x, [-1, WIDTH, HEIGHT, 1])

# Weights and biases between input and hidden layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Hidden layer 1
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, stride_size=4) + b_conv1)
# Max pooled hidden layer 1
h_pool1 = max_pool_2x2(h_conv1) #dim is 80x80x32

print('h_pool1', h_pool1.shape)

# Weights and biases between hidden layer 1 and 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = weight_variable([64])

# Hidden layer 2
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, stride_size=4) + b_conv2)
# Max pooled hidden layer 2
h_pool2 = max_pool_2x2(h_conv2) # dim is 20x20x64

print('h_pool2', h_pool2.shape)

# Weights and biases between hidden layer 2 and 3
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

# Hidden layer 3
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
# Max pooled hidden layer 3
h_pool3 = max_pool_2x2(h_conv3) # dim is 5x5x128

print('h_pool3', h_pool3.shape)

In [ ]:
# Dense layer 1
W_d1 = weight_variable([3*3*128, 512])
b_d1 = weight_variable([512])

h_pool3_flat = tf.reshape(h_pool3, [-1, 3*3*128])
h_d1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_d1) + b_d1)

keep_prob = tf.placeholder(tf.float32)
h_d1_drop = tf.nn.dropout(h_d1, keep_prob)

# Dense layer 2
W_d2 = weight_variable([512, 1])
b_d2 = bias_variable([1])

# Output
y_pred = tf.matmul(h_d1_drop, W_d2) + b_d2

In [ ]:
mse = tf.losses.mean_squared_error(y, y_pred)
train_step = tf.train.GradientDescentOptimizer(1e-7).minimize(mse)
init = tf.global_variables_initializer()

In [275]:
with tf.Session() as sess:
    sess.run(init)
    for n in range(1000):
        sess.run(train_step, feed_dict={
            x: im_array,
            y: np.array([[105]]),
            keep_prob: 0.5
        })
        print(sess.run([mse], feed_dict={
            x: im_array,
            y: np.array([[105]]),
            keep_prob: 1
        }))